In [10]:
import requests
import pandas as pd
from lxml import etree

In [11]:
api_key = "577df7196ceeb3a853b9aac5b7bfe5640209"           #You can remove it from the parameters but it is recommended to have it and to increase your request rate from 3 to 10 per second

def search_pubmed(query):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": "10000",
        "usehistory": "y",
        "email": "nawar82@gmail.com",
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        root = etree.fromstring(response.content)
        id_list = root.xpath('//IdList/Id')
        return [id.text for id in id_list]
    else:
        print("Error occurred while searching")
        return []

def fetch_abstracts(id_list):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    ids = ','.join(id_list)
    params = {
        "db": "pubmed",
        "retmode": "xml",
        "id": ids,
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    abstracts = []
    if response.status_code == 200:
        root = etree.fromstring(response.content)
        articles = root.xpath('//PubmedArticle')
        for article in articles:
            pmid = article.find('.//PMID').text
            abstract_text = article.find('.//Abstract/AbstractText')
            if abstract_text is not None:
                abstracts.append({"PMID": pmid, "Abstract": abstract_text.text})
    else:
        print("Error occurred while fetching details")
    return abstracts


In [12]:
# Example usage
query = "tsh AND Mendelian randomization"
id_list = search_pubmed(query)
abstracts = fetch_abstracts(id_list)

# Convert list of abstracts to DataFrame
df_abstracts = pd.DataFrame(abstracts)

df_abstracts


,PMID,Abstract
0,38405140,Previous studies have suggested a potential as...
1,38392839,The association between thyroid function and v...
2,38375193,To determine whether there is a causal relatio...
3,38368359,The role of thyroid health in temporomandibula...
4,38355654,Genome-wide association studies have reported ...
...,...,...
62,30248900,The thyroid plays a key role in development an...
63,30016786,Increasing evidence suggests an association be...
64,29544020,"With population aging, prevalence of low bone ..."
65,28819171,To clarify the role of thyroid function in isc...


In [1]:
df_abstracts.info()

NameError: name 'df_abstracts' is not defined

# Test